# Domain Definitions

In [216]:
import pandas as pd
from dataclasses import dataclass
from typing import Optional, List, Set, Sequence, Dict, Tuple

In [217]:
@dataclass(frozen=True)
class Item:
    name: str
    value: float
    weight: int
    time_to_steal: int

In [218]:
@dataclass(frozen=False)
class Node:
    name: str
    items: Set[Item]

    def __init__(self, name: str):
        self.name = name
        self.items = set()

    def add_item(self, item: Item):
        self.items.add(item)
        return self

    def add_items(self, items: Sequence['Item']):
        [self.add_item(item) for item in items]
        return self

    def __hash__(self) -> int:
        return self.name.__hash__()

In [219]:
@dataclass(frozen=True)
class Edge:
    source: Node
    destination: Node
    travelling_time: int
    travelling_cost: float

In [220]:
class Graph:
    nodes: Dict[str,Node]
    edges: Dict[Tuple[Node,Node], Edge]
    connections: Dict[Node, List[Edge]]

    def __init__(self):
        self.nodes = dict()
        self.edges = dict()
        self.connections = dict()

    def get_node(self, name: str) -> Optional[Node]:
        return self.nodes.get(name)

    def get_nodes(self) -> List[Node]:
        return list(self.nodes.values())

    def get_edges(self) -> List[Edge]:
        return list(self.edges.values())

    def get_connections_from_node(self, node: Node) -> List[Edge]:
        return self.connections.get(node, [])

    def get_connections_from(self, node_name: str) -> List[Edge]:
        node: Optional[Node] = self.get_node(node_name)
        return self.get_connections_from_node(node)

    def add_edge(self, src: str, dst: str, tt: int, tc: float) -> 'Graph':
        source = Node(src)
        destination = Node(dst)
        edge = Edge(source, destination, travelling_time=tt, travelling_cost=tc)
        self._add_node(source)._add_node(destination)
        self.edges[(source, destination)] = edge
        self.connections[source] = self.connections.get(source, []) + [edge]
        return self

    def add_node(self, node_name: str) -> 'Graph':
        self.nodes[node_name] = Node(node_name)
        return self

    def _add_node(self, node: Node) -> 'Graph':
        self.nodes[node.name] = node
        return self

# Graph Construction

# Problem Solving 